In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import ta, pickle, json, time

In [18]:
df = pd.read_csv("data/train.csv", 
                 names=['date', 'open', 'high', 'low', 'close', 'volume'])
df

,date,open,high,low,close,volume
0,2021-01-01 00:00:00,28923.63,28961.66,28913.12,28961.66,0.0
1,2021-01-01 00:01:00,28961.67,29017.50,28961.01,29009.91,0.0
2,2021-01-01 00:02:00,29009.54,29016.71,28973.58,28989.30,0.0
3,2021-01-01 00:03:00,28989.68,28999.85,28972.33,28982.69,0.0
4,2021-01-01 00:04:00,28982.67,28995.93,28971.80,28975.65,0.0
...,...,...,...,...,...,...
84876,2021-02-28 23:55:00,44883.62,44996.28,44873.45,44926.12,0.0
84877,2021-02-28 23:56:00,44932.68,45037.55,44912.24,45035.99,0.0
84878,2021-02-28 23:57:00,45029.75,45050.00,44973.17,45023.35,0.0
84879,2021-02-28 23:58:00,45023.36,45071.42,44991.36,45063.42,0.0


In [19]:
def get_features(df):
    df_ = df[["open", "high", "low", "close"]]
    # adding RSI
    df_["rsi"] = ta.momentum.rsi(df.close, fillna=True)/100
    # adding StochasticOscillator
    stoch = ta.momentum.StochasticOscillator(high=df.high, low=df.low, close=df.close, fillna=True)
    df_["stoch"] = stoch.stoch() / 100
    df_["stoch_signal"] = stoch.stoch_signal() / 100
    # adding Aroon
    aroon = ta.trend.AroonIndicator(df.close, fillna=True)
    df_["aroon"] = (aroon.aroon_indicator() + 100) / 200
    df_["aroon_down"] = aroon.aroon_down() / 100
    df_["aroon_up"] = aroon.aroon_up() / 100
    
    return df_

# GA

## Create Population

In [20]:
# indiv shape (6, 2, 2)
n_indicator = 6 # 6 indicators + stoploss take profit
n_action = 2
n_borne = 2
def conform_indiv(indiv):
    for indicator in range(n_indicator+1): # 6 indicators + stoploss take profit
        for action in range(n_action):
            inf_index = (indicator, action, 0)
            sup_index = (indicator, action, 1)
            if indiv[inf_index] > indiv[sup_index]:
                indiv[inf_index], indiv[sup_index] = indiv[sup_index], indiv[inf_index]
    for borne in range(n_borne):
        for action in range(n_action):
            index = (-1, action, borne)
            indiv[index] = (indiv[index] + borne)/2
    return indiv

def create_indiv():
    indiv = np.random.random((7, 2, 2))
    indiv = conform_indiv(indiv)
    return indiv

In [21]:
def create_pop(size=100):
    pop = [(None, create_indiv(), 0) for i in range(size)]
    return pop
        

##  Evaluation

In [22]:
spread = 30
indicators = ['rsi', 'stoch', 'stoch_signal', 'aroon', 'aroon_down', 'aroon_up']

def match_condition(indiv, df):
    for i in range(n_indicator):
        inf = indiv[i, 0]
        sup = indiv[i, 1]
        value = df[indicators[i]]
        if value < inf or value > sup:
            return False
    return True

def trade(df, low, high, start):
    ls = df.values.tolist()
    n = len(ls)
    for i in range(n):
        if ls[i][0] > high: # test high
            return high, i
        if ls[i][1] < low: # test low
            return low, i
    return start, n

def eval_indiv(indiv, df):
    n = len(df)
    score = 0
    i = 15
    while i < n-15:
        df_indicators = df.iloc[i][indicators]
        price = df.iloc[i]["close"]
        if match_condition(indiv[:, 0], df_indicators): # Test achat
            out, d = trade(df.iloc[i:][["high", "low"]], 
                            price * (indiv[-1, 0, 0]+.5),
                            price * (indiv[-1, 0, 1]+.5),
                            price
                           )
            score += (out-price) - spread
            i += d
        if match_condition(indiv[:, 1], df_indicators): # Test vente
            out, d = trade(df.iloc[i:][["high", "low"]], 
                            price * (indiv[-1, 1, 0]+.5),
                            price * (indiv[-1, 1, 1]+.5),
                            price
                           )
            score += (price-out) - spread
            i += d
        i+=1
    return score

# Selection

In [23]:
def crossover(indiv1, indiv2, rate=.5):
    indiv1 = indiv1.reshape((((n_indicator+1)*n_action, n_borne)))
    indiv2 = indiv2.reshape((((n_indicator+1)*n_action, n_borne)))
    indiv = []
    for i in range((n_indicator+1)*n_action):
        if np.random.random() > rate:
            indiv.append(indiv1[i])
        else:
            indiv.append(indiv2[i])
    indiv = np.array(indiv).reshape((n_indicator+1, n_action, n_borne))
    return indiv
    
def create_cross_pop(ls, size):
    ls_ = []
    n = len(ls)
    for i in range(size):
        a, b = np.random.randint(n, size=2)
        ls_.append((None, crossover(ls[a][1], ls[b][1]), 0))
    return ls_
    
def mutation(ls, rate=.3):
    for i in range(len(ls)):
        indiv = ls[i][1]
        indiv_ = create_indiv()
        indiv = crossover(indiv, indiv_, rate)
        ls[i] = (None, indiv, 0)
    return ls

def new_gen(ls):
    ls_ = []
    best_times = 0
    for (score, indiv, times) in ls[:10]: # taille 10
        if score > 0:
            times += 1
        best_times = max(best_times, times)
        ls_.append((score, indiv, times+1))
    ls_ = ls_ + mutation(ls_) # taille 10+10 = 20
    ls_ = ls_ + create_cross_pop(ls[:30], 50) # taille 20+50 = 70
    ls_ = ls_ + create_pop(30) # taille 70+30 = 100
    return ls_, best_times

# Processus

In [24]:
df = get_features(df)
df

<ipython-input-19-c0a76c8ba45e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_["rsi"] = ta.momentum.rsi(df.close, fillna=True)/100


,open,high,low,close,rsi,stoch,stoch_signal,aroon,aroon_down,aroon_up
0,28923.63,28961.66,28913.12,28961.66,1.000000,1.000000,1.000000,0.50,0.04,0.04
1,28961.67,29017.50,28961.01,29009.91,1.000000,0.927285,0.963642,0.52,0.04,0.08
2,29009.54,29016.71,28973.58,28989.30,0.684928,0.729833,0.885706,0.52,0.04,0.08
3,28989.68,28999.85,28972.33,28982.69,0.617707,0.666507,0.774542,0.52,0.04,0.08
4,28982.67,28995.93,28971.80,28975.65,0.555209,0.599061,0.665134,0.52,0.04,0.08
...,...,...,...,...,...,...,...,...,...,...
84876,44883.62,44996.28,44873.45,44926.12,0.365155,0.169159,0.144865,0.04,0.96,0.04
84877,44932.68,45037.55,44912.24,45035.99,0.436746,0.413320,0.218912,0.06,0.92,0.04
84878,45029.75,45050.00,44973.17,45023.35,0.430728,0.385231,0.322570,0.08,0.88,0.04
84879,45023.36,45071.42,44991.36,45063.42,0.456303,0.478853,0.425801,0.16,0.84,0.16


In [25]:
list_df = []
delta = 2120
for i in range(40):
    list_df.append(df.iloc[80+i*delta:80+(i+1)*delta:])
len(list_df)

40

In [65]:
def unarray(array):
    result = []
    for i in range(len(array)):
        values = array[i]
        result.append((values[0], values[1].tolist(), values[2]))
    return result

In [67]:
best_score = -1
best_times = 0
gen = 1
population = create_pop()
liste_best = []

while best_times < 40:
    df_ = list_df[gen%40]
    for i in range(len(population)):
        population[i] = (eval_indiv(population[i][1], df_), population[i][1], population[i][2])
    population.sort(key=lambda x: x[0], reverse=True)
    best_score = population[0][0]
    lowest_score = population[-1][0]
    liste_best.append(best_score)
    
    temp = unarray(population)
    json.dump(temp[0], open(f"record/{gen}.json", "w"))
    json.dump(temp, open(f"record/actual.json", "w"))
    population, best_times = new_gen(population)
    print(f"Generation {gen} best_score : {best_score}")
    print(f"Times : {best_times}")
    print(f"lowest_score : {lowest_score}")
    gen += 1
    time.sleep(5)

Generation 1 best_score : 0
Times : 0
lowest_score : -30.0
Generation 2 best_score : 0
Times : 0
lowest_score : -3001.535823083388
Generation 3 best_score : 0
Times : 0
lowest_score : -30.0
Generation 4 best_score : 0
Times : 0
lowest_score : -3187.0380721931797
Generation 5 best_score : 0
Times : 0
lowest_score : -30.0
Generation 6 best_score : 0
Times : 0
lowest_score : -8136.3553725112
Generation 7 best_score : 0
Times : 0
lowest_score : -2481.3940843083546
Generation 8 best_score : 0
Times : 0
lowest_score : -2526.7171114062585
Generation 9 best_score : 0
Times : 0
lowest_score : -4584.543943905192
Generation 10 best_score : 0
Times : 0
lowest_score : -30.0
Generation 11 best_score : 0
Times : 0
lowest_score : -30.0
Generation 12 best_score : 0
Times : 0
lowest_score : -30.0
Generation 13 best_score : 0
Times : 0
lowest_score : -30.0
Generation 14 best_score : 0
Times : 0
lowest_score : -30.0
Generation 15 best_score : 0
Times : 0
lowest_score : -30.0
Generation 16 best_score : 0
T

KeyboardInterrupt: 

In [66]:
temp = unarray(population)
json.dump(temp[0], open(f"record/{gen}.json", "w"))
json.dump(temp, open(f"record/actual.json", "w"))

In [62]:
temp

[[0,
  array([[[0.42947249, 0.87222359],
          [0.00852445, 0.58057268]],
  
         [[0.40215551, 0.47525154],
          [0.42048782, 0.87676034]],
  
         [[0.62680327, 0.70567457],
          [0.07928321, 0.74364249]],
  
         [[0.7635654 , 0.9454976 ],
          [0.15906039, 0.19744593]],
  
         [[0.24314256, 0.74768589],
          [0.44298847, 0.98698   ]],
  
         [[0.61208243, 0.62540287],
          [0.43479358, 0.93902795]],
  
         [[0.4516605 , 0.97322841],
          [0.28495242, 0.83164267]]]),
  0],
 [0,
  array([[[0.67421216, 0.94990979],
          [0.65608199, 0.9701641 ]],
  
         [[0.51978883, 0.99474515],
          [0.32411734, 0.8260382 ]],
  
         [[0.56919073, 0.7817617 ],
          [0.06246408, 0.29015523]],
  
         [[0.4259078 , 0.47728685],
          [0.22728492, 0.37836679]],
  
         [[0.02118308, 0.48542675],
          [0.21877809, 0.61469389]],
  
         [[0.09715056, 0.66073255],
          [0.44061724, 0.56261012]],


# Testing result

In [21]:
population

[(None,
  array([[[0.06332101, 0.31679554],
          [0.43225743, 0.799737  ]],
  
         [[0.37645825, 0.77380917],
          [0.51424074, 0.95560542]],
  
         [[0.86371351, 0.97110912],
          [0.54066928, 0.80679498]],
  
         [[0.19311894, 0.26892905],
          [0.23783564, 0.39793843]],
  
         [[0.13053637, 0.65249854],
          [0.20760058, 0.47130995]],
  
         [[0.16493456, 0.92553371],
          [0.63830879, 0.76348641]],
  
         [[0.37088747, 0.96253349],
          [0.33185917, 0.91444379]]])),
 (None,
  array([[[0.27115222, 0.42958057],
          [0.54857296, 0.8504211 ]],
  
         [[0.00769851, 0.68297616],
          [0.13146086, 0.46383991]],
  
         [[0.79036061, 0.99519789],
          [0.30002364, 0.74029509]],
  
         [[0.05268712, 0.42807813],
          [0.04751869, 0.41931972]],
  
         [[0.24753915, 0.37807649],
          [0.47746342, 0.48232637]],
  
         [[0.35979219, 0.54827751],
          [0.12846606, 0.32572002]],

In [22]:
with open(f"record/actual", "rb") as fp:   #Pickling
    population = pickle.load(fp)
population

[(6733.575940218907,
  array([[[0.02914264, 0.73032576],
          [0.1160545 , 0.51305172]],
  
         [[0.37645825, 0.77380917],
          [0.51424074, 0.95560542]],
  
         [[0.86371351, 0.97110912],
          [0.71802007, 0.86029045]],
  
         [[0.4410307 , 0.48461594],
          [0.23783564, 0.39793843]],
  
         [[0.13053637, 0.65249854],
          [0.20760058, 0.47130995]],
  
         [[0.16493456, 0.92553371],
          [0.00131751, 0.39252566]],
  
         [[0.14901292, 0.95109901],
          [0.33185917, 0.91444379]]])),
 (5924.485530752594,
  array([[[0.27115222, 0.42958057],
          [0.15865859, 0.47277849]],
  
         [[0.00769851, 0.68297616],
          [0.47547668, 0.93030657]],
  
         [[0.28030837, 0.35709158],
          [0.30002364, 0.74029509]],
  
         [[0.35590118, 0.89990594],
          [0.04751869, 0.41931972]],
  
         [[0.24753915, 0.37807649],
          [0.47746342, 0.48232637]],
  
         [[0.81010846, 0.85212204],
          

In [23]:
indiv = population[0][1]

In [26]:
def trade(df, low, high, start):
    ls = df.values.tolist()
    for i in range(len(ls)):
        if ls[i][0] > high: # test high
            print(i)
            return high
        if ls[i][1] < low: # test low
            print(i)
            return low
    return start


def eval_indiv_test(indiv, df):
    n = len(df)
    score = 0
    ordre = [[], []]
    for i in range(15, n-15):
        df_indicators = df.iloc[i][indicators]
        if match_condition(indiv[:, 0], df_indicators): # Test achat
            print("ACHAT")
            price = df.iloc[i]["close"]
            ordre[0].append(i)
            ordre[1].append(price)
            out = trade(df.iloc[i:][["high", "low"]], 
                        price * (indiv[-1, 0, 0]+.5),
                        price * (indiv[-1, 0, 1]+.5),
                        price
                       )
            score += (out-price) - spread
        
        if match_condition(indiv[:, 1], df_indicators): # Test vente
            print("VENTE")
            price = df.iloc[i]["close"]
            ordre[0].append(i)
            ordre[1].append(price)
            out = trade(df.iloc[i:][["high", "low"]], 
                        price * (indiv[-1, 1, 0]+.5),
                        price * (indiv[-1, 1, 1]+.5),
                        price
                       )
            score += (price-out) - spread
    return score, ordre
scores, orders = [], []
for i in range(40):
    print(i)
    score, ordre = eval_indiv_test(indiv, list_df[i])
    scores.append(score)
    orders.append(ordre)

0
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
1
VENTE
VENTE
2
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
3
VENTE
VENTE
VENTE
VENTE
VENTE
4
VENTE
VENTE
5
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
6
VENTE
1876
VENTE
1874
VENTE
1517
VENTE
1515
VENTE
1183
VENTE
1008
VENTE
7
VENTE
VENTE
8
VENTE
VENTE
9
VENTE
VENTE
10
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
11
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
12
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
13
VENTE
VENTE
VENTE
VENTE
VENTE
14
VENTE
VENTE
VENTE
15
VENTE
VENTE
16
ACHAT
VENTE
17
VENTE
VENTE
VENTE
VENTE
VENTE
18
VENTE
VENTE
VENTE
19
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
20
VENTE
VENTE
VENTE
VENTE
21
VENTE
VENTE
22
VENTE
VENTE
VENTE
23
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
24
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
25
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VENTE
VE

In [28]:
indiv

array([[[0.02914264, 0.73032576],
        [0.1160545 , 0.51305172]],

       [[0.37645825, 0.77380917],
        [0.51424074, 0.95560542]],

       [[0.86371351, 0.97110912],
        [0.71802007, 0.86029045]],

       [[0.4410307 , 0.48461594],
        [0.23783564, 0.39793843]],

       [[0.13053637, 0.65249854],
        [0.20760058, 0.47130995]],

       [[0.16493456, 0.92553371],
        [0.00131751, 0.39252566]],

       [[0.14901292, 0.95109901],
        [0.33185917, 0.91444379]]])

In [27]:
scores

[-240.0,
 -60.0,
 -240.0,
 -150.0,
 -60.0,
 -330.0,
 40797.072210767736,
 -60.0,
 -60.0,
 -60.0,
 -360.0,
 -240.0,
 -240.0,
 -150.0,
 -90.0,
 -60.0,
 -60.0,
 -150.0,
 -90.0,
 -270.0,
 -120.0,
 -60.0,
 -90.0,
 -360.0,
 -300.0,
 -450.0,
 -180.0,
 -150.0,
 -270.0,
 -60.0,
 -480.0,
 -480.0,
 -150.0,
 -120.0,
 -270.0,
 86728.76967440473,
 -60.0,
 -120.0,
 -240.0,
 -360.0]

In [308]:
len(ordre[0])

118

In [310]:
indiv

array([[[5.37439555e-03, 7.67343265e-01],
        [1.98508601e-02, 6.37935004e-01]],

       [[3.22765232e-02, 9.95086903e-01],
        [3.57818550e-01, 6.60599174e-01]],

       [[7.98987501e-04, 8.30657881e-01],
        [3.35623964e-01, 6.79689057e-01]],

       [[1.55014361e-01, 9.79410228e-01],
        [1.75791397e-02, 2.29564062e-01]],

       [[1.25399350e-02, 7.32196314e-01],
        [6.40864664e-01, 9.61259592e-01]],

       [[5.07000530e-02, 9.68362793e-01],
        [8.90148648e-03, 2.60082838e-01]],

       [[9.05290077e-01, 9.41884512e-01],
        [1.56291855e-01, 2.25117920e-01]]])